In [1]:
!pip install dask[complete] --upgrade --quiet
!pip install aiohttp --quiet

     |████████████████████████████████| 890kB 8.1MB/s 
     |████████████████████████████████| 675kB 19.7MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
ERROR: distributed 2021.1.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
     |████████████████████████████████| 1.3MB 8.0MB/s 
     |████████████████████████████████| 296kB 34.4MB/s 
     |████████████████████████████████| 143kB 32.9MB/s 


In [5]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, progress

In [3]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:37629 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [8]:
%%time
x = da.random.random((10000, 10000), chunks=(1000, 1000))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 323 ms, sys: 38.9 ms, total: 362 ms
Wall time: 1.73 s


In [9]:
%%time
x = np.random.random((10000, 10000))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)

CPU times: user 1.73 s, sys: 13.1 ms, total: 1.75 s
Wall time: 1.67 s


In [10]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))

In [11]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 431 ms, sys: 32.8 ms, total: 463 ms
Wall time: 1.8 s


In [12]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))
# This persists the `x` array into the memory
x.persist()

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [13]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 216 ms, sys: 35.6 ms, total: 251 ms
Wall time: 923 ms


Persisting is even faster if you have the ram space.

## Change the code above by setting chunks=(250, 250). How long does it take to run?

In [14]:
%%timeit
x = da.random.random((10000, 10000), chunks=(250, 250))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

1 loop, best of 3: 5.97 s per loop


It took almost 6 seconds to run.

## Now, set the parameter to chunks=(500, 500). How long does it take to run? Does this one or the previous one run more quickly? Why?

In [15]:
%%timeit
x = da.random.random((10000, 10000), chunks=(500, 500))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

1 loop, best of 3: 2.55 s per loop


It took only 2.55 seconds to run. It was much faster than the 250x250 chunks approach. This is because we are taking a bigger chunk of the data to iterate over, which means fewer number of chunks required than the 250x250 approach. So with bigger chunks their are fewer iterations, which means faster run times.